## ECI 249 HW \#5

Kenneth Larrieu

### a) Formulate this problem as an optimization problem, minimizing the net expected value cost of managing the floodplain costs and damage. 

The optimal floodplain management options may be found by minimizing the net expected value cost for floodplain management. The next expected value cost is:

$$EVC = \sum_{i=1}^3 c_{P_{i}} X_{P_i} + \sum_{s=1}^{5} p(s) \left( \sum_{j=1}^3 c_{E_{js}} X_{E_{js}} + D_s \right)$$

where 

$X_{P_i}$ is the number of units used for permanent floodplain management option $P_i$

$c_{P_{i}}$ is the annualized unit cost of the option

$p(s)$ is the probability of flow range $s$

$X_{E_{js}}$ is the number of units used for emergency floodplain management option $E_{js}$ at flow range $s$

$c_{E_{js}}$ is the unit cost of that option, and

$D_s$ is the damage incurred by flood s given the applied floodplain management options.

Thus, we want to find the vector $X_{P_i}$ of permanent option quantities and the tensor $X_{E_{js}}$ of temporary option quantities at each discharge range which minimize $EVC$, subject to the imposed bounds on values of their elements.

### b) Minimize the expected value cost of flood damage and control costs by selecting which permanent and short-term flood control measures should be undertaken.  Do this for both current conditions and with the upstream project.

In [25]:
import numpy as np
import pandas as pd

def dot(l1, l2):
    """Dot product between two lists"""
    return sum([e1 * e2 for e1, e2 in zip(l1, l2)])

class LP:
    def __init__(self, *args):
        self.q_names = ['< 5,000 cfs', '5-6,000 cfs', '6-8,000 cfs', '8-10,000 cfs', '10,000+ cfs']
        self.q_probs = [0.8, 0.11, 0.06, 0.02, 0.01]
        if 'upstream' in args:
            self.q_probs = [0.9, 0.05, 0.03, 0.01, 0.01]
        self.damage0 = [0, 2.1e6, 3e6, 4.2e6, 6e6]

        self.perm_names = ['Raise Structures', 'Warning System', 'Sacrificial First Stories']
        self.perm_costs = [10, 1, 40]
        self.perm_lims = [1e6, 200e3, 200e3]
        self.perm_reds = [[0, 100, 70, 60, 10], [0, 2, 3, 4, 7], [0, 100, 60, 50, 20]]
        self.perm_reds = [[0, 200, 90, 70, 10], [0, 3, 4, 6, 10], [0, 100, 60, 50, 20]] # ***

        self.em_names = ['Evacuate', 'Sandbagging', 'Heightened Levee Monitoring']
        self.em_costs = [200e3, 20e3, 1]
        self.em_costs = [100e3, 30e3, 1] # ***
        self.em_lims = [1, 2, 20e3]
        self.em_reds = [[0, 200e3, 300e3, 500e3, 1e6], [0, 1e6, 800e3, 0, 0], [0, 2, 1, 0, 0]]

    def damage(self, q, perm_vector, em_vector):
        """
        For flow index q and vectors for permanent option values and emergency option values,
        return the associated damage
        """
        d0 = self.damage0[q]
        perm_red = dot(perm_vector, [self.perm_reds[option][q] for option in range(len(self.perm_names))])
        em_red = dot(em_vector, [self.em_reds[option][q] for option in range(len(self.em_names))])

        return max(d0 - perm_red - em_red, 0)

    def evc(self, perm_vector, em_vector):
        """EVC given perm_vector and em_vector (tensor const. w discharge)"""
        perm_cost = dot(perm_vector, self.perm_costs)
        em_cost = dot(em_vector, self.em_costs)
        dam_cost = dot(self.q_probs, [self.damage(q, perm_vector, em_vector) for q in range(len(self.q_names))])

        return perm_cost + em_cost + dam_cost

    def evc_true(self, perm_vector, em_tensor):
        perm_cost = dot(perm_vector, self.perm_costs)
        p_costs = 0
        for q in range(len(self.q_names)):
            p_costs += self.q_probs[q] * dot(self.em_costs, em_tensor[q])
            p_costs += self.q_probs[q] * self.damage(q, perm_vector, em_tensor[q])
        total = perm_cost + p_costs
        return total
    
    def run_LP(self):
        
        # get optimal values of each permanent option
        perm_vals = []
        # for each perm choice
        for option in range(len(self.perm_names)):
            best_option_val = None
            best_cost = np.inf
            perm_vector = [0] * len(self.perm_names)
            em_tensor = [[0] * len(self.em_names)] * len(self.q_names)
            # iterate over possible values, save best
            for option_val in range(int(self.perm_lims[option])+1):
                perm_vector[option] = option_val
                cost = self.evc_true(perm_vector, em_tensor)
                if cost < best_cost:
                    best_cost = cost
                    best_option_val = option_val
                    
            perm_vals.append(best_option_val)

        # get optimal values of each emergency option at each discharge
        em_vals = []
        # for each discharge
        for q in range(len(self.q_names)):
            # for each emergency choice
            em_vals_q = []
            for option in range(len(self.em_names)):
                best_option_val = None
                best_cost = np.inf
                perm_vector = [0] * len(self.perm_names)
                em_tensor = [[0] * len(self.em_names)] * len(self.q_names)
                # iterate over possible values, save best
                for option_val in range(int(self.em_lims[option])+1):
                    em_tensor[q][option] = option_val
                    cost = self.evc_true(perm_vector, em_tensor)
                    if cost < best_cost:
                        best_cost = cost
                        best_option_val = option_val

                em_vals_q.append(best_option_val)

            em_vals.append(em_vals_q)
        
        self.optimal_evc = self.evc_true(perm_vals, em_vals)
        
        perm_df = pd.DataFrame(perm_vals, index=self.perm_names, columns=[r'$Optimal X_{P_i}$'])
        perm_df.index.name = 'Permanent Management Option'
        em_df = pd.DataFrame(em_vals, index=self.q_names, columns=self.em_names)
        em_df.index.name = 'Flood Discharge Range'

        return perm_df, em_df

In [26]:
lp_1 = LP()
perm_df, em_df = lp_1.run_LP()

In [27]:
lp_1.evc_true([500, 0, 0], [[0, 0, 0],[0, 2, 0],[1, 2, 20e3],[1, 0, 0],[1, 0, 0]])

210750.0

In [28]:
lp_1.optimal_evc

320550.0

In [29]:
perm_df

,$Optimal X_{P_i}$
Permanent Management Option,
Raise Structures,10500
Warning System,0
Sacrificial First Stories,0


In [30]:
em_df

,Evacuate,Sandbagging,Heightened Levee Monitoring
Flood Discharge Range,,,
"< 5,000 cfs",0,2,0
"5-6,000 cfs",0,2,0
"6-8,000 cfs",0,2,0
"8-10,000 cfs",0,2,0
"10,000+ cfs",0,2,0


In [5]:
# now for the upstream section
lp_2 = LP("upstream")
perm_df, em_df = lp_2.run_LP()

In [6]:
perm_df

,$Optimal X_{P_i}$
Permanent Management Option,
Raise Structures,0
Warning System,0
Sacrificial First Stories,0


In [7]:
em_df

,Evacuate,Sandbagging,Heightened Levee Monitoring
Flood Discharge Range,,,
"< 5,000 cfs",0,0,0
"5-6,000 cfs",0,2,20000
"6-8,000 cfs",1,2,0
"8-10,000 cfs",1,0,0
"10,000+ cfs",1,0,0


### c) What is the value of the upstream project, in terms of flood damage reduction?

Optimal EVC for downstream - optimal EVC for upstream

### d) For each used and unused decision, what would be the range of unit costs for which these optimized decisions would not change?  Present this as a table.

asdf